In [68]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import CountVectorizer

In [69]:
X = [[0., 0.], [1., 1.]]

In [70]:
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)  

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [71]:
clf.predict([[-5, 1], [-13., -2.]])

array([0, 0])

In [72]:
[coef.shape for coef in clf.coefs_]

[(2, 5), (5, 2), (2, 1)]

In [73]:
dfStockData = pd.read_json("../Dropbox/Project/TSLA_stocks.json").transpose()
dfStockData.head()

,1. open,2. high,3. low,4. close,5. volume
2010-06-29,19.00,25.0000,17.54,23.89,18766300.0
2010-06-30,25.79,30.4192,23.30,23.83,17187100.0
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0
2010-07-02,23.00,23.1000,18.71,19.20,5139800.0
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0


In [80]:
df = json.load(open("../Dropbox/Project/messages/resp_0.json"))
dfTweet = json_normalize(df['messages'])
dfTweet = dfTweet[['body', 'created_at']]
dfTweet.head()

,body,created_at
0,"$TSLA Don&#39;t forget, Elon has STUCK to his ...",2018-10-31T22:08:59Z
1,@posicaprinia I like your deattached garage......,2018-10-31T22:08:47Z
2,"$TSLA right before close today, someone made a...",2018-10-31T22:08:01Z
3,Added back a second planned short leg $TSLA at...,2018-10-31T22:07:33Z
4,$TSLA lol why the heavy selling in AH? Can som...,2018-10-31T22:06:16Z


In [88]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit_transform(dfTweet['body']).todense()
vectorizer.vocabulary_
print(max(vectorizer.vocabulary_))

years
